In [1]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [2]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

for i in range(len(files)):
    if 'no' in files[i]:
        if '__' not in files[i]:
            if 'all' not in files[i]:
                filenames.append(files[i])
                print(i, files[i])

17 Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj
21 Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj
25 Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj
31 Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj
35 Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj
39 Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj
45 Cu_100_u2_RPBE_no001_CO_n1_d8.traj
49 Cu_100_u2_RPBE_no010_CO_n1_d8.traj
53 Cu_100_u2_RPBE_no020_CO_n1_d8.traj
59 Cu_100_u2_optB88-vdW_no001_CO_n1_d8.traj
63 Cu_100_u2_optB88-vdW_no010_CO_n1_d8.traj
67 Cu_100_u2_optB88-vdW_no020_CO_n1_d8.traj
74 Cu_100_u2_vdW-DF2_no001_CO_n1_d8.traj
78 Cu_100_u2_vdW-DF2_no010_CO_n1_d8.traj
82 Cu_100_u2_vdW-DF2_no020_CO_n1_d8.traj
87 Cu_100_u2_vdW-DF_no001_CO_n1_d8.traj
91 Cu_100_u2_vdW-DF_no010_CO_n1_d8.traj
95 Cu_100_u2_vdW-DF_no020_CO_n1_d8.traj
101 Cu_111_u2_BEEF-vdW_no001_CO_n1_d8.traj
105 Cu_111_u2_BEEF-vdW_no008_CO_n1_d8.traj
109 Cu_111_u2_BEEF-vdW_no015_CO_n1_d8.traj
113 Cu_111_u2_BEEF-vdW_no020_CO_n1_d8.traj
119 Cu_111_u2_RPBE-D2_no001_CO_n1_d8.traj
123 Cu_111_u2_RPBE-D2_no008_CO_n1_d8.traj
127 Cu

In [3]:
for file in filenames:
    try:
        A = make_database(file)
        A.delete_from_database()
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE_no001_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE_no010_CO_n1_d8.traj deleted
-----------------------------------------------------------
Cu_100_u2_RPBE_no020_CO_n1_d8.traj deleted
----------------------------------------------------

In [5]:
for file in filenames:
    try:
        A = make_database(file)
    except:
        print(file, 'Some error occured.')
    A.add_to_database()
    A.update_database_Eeach()
    A.check_database()
    

-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
----------------------------------

-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
-----------------------------------------------------------
Already in database
This function is for surface with more than 2 adsorbates.
----------------------------------

In [6]:
for file in filenames:
    try:
        A = make_database(file)
        data = A.check_database()
        print(data['name'], data['converged'], data['not_moved'], data['kept_sites'], data['isvalid'], data['totaladsE'])
    except:
        print(file, 'Some error occured.')

-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj no yes yes no -0.59465535
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj yes yes yes yes -0.3970746700000021
-----------------------------------------------------------
Cu_100_u2_BEEF-vdW_no019_CO_n1_d8.traj yes yes yes yes -0.5161128500000025
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj no yes yes no -0.7787489500000078
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no010_CO_n1_d8.traj yes yes yes yes -0.7644200699999999
-----------------------------------------------------------
Cu_100_u2_RPBE-D2_no020_CO_n1_d8.traj yes yes yes yes -0.698101390000005
-----------------------------------------------------------
Cu_100_u2_RPBE_no001_CO_n1_d8.traj yes yes yes yes -0.5497481199999967
-----------------------------------------------------------
Cu_100_u2_RPBE_

-----------------------------------------------------------
Pt_111_u2_RPBE_no008_CO_n1_d9.traj yes yes yes yes -1.3836878299999995
-----------------------------------------------------------
Pt_111_u2_RPBE_no015_CO_n1_d9.traj yes yes yes yes -1.3758159500000033
-----------------------------------------------------------
Pt_111_u2_RPBE_no020_CO_n1_d9.traj yes yes yes yes -1.3621210200000036
-----------------------------------------------------------
Pt_111_u2_optB88-vdW_no001_CO_n1_d9.traj yes yes yes yes -1.6671941999999973
-----------------------------------------------------------
Pt_111_u2_optB88-vdW_no008_CO_n1_d9.traj yes yes yes yes -1.7612890099999987
-----------------------------------------------------------
Pt_111_u2_optB88-vdW_no015_CO_n1_d9.traj yes yes yes yes -1.7439587900000006
-----------------------------------------------------------
Pt_111_u2_optB88-vdW_no020_CO_n1_d9.traj yes yes yes yes -1.762805819999997
-----------------------------------------------------------


In [11]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         A.update_database_adsorbates_correlation()
#     except:
#         print(file, 'Some error occured.')

In [22]:
# for file in filenames:
#     try:
#         A = make_database(file)
#         print(A.check_database()['minimum_distance'])
#     except:
#         print(file, 'Some error occured.')

In [15]:
file = 'Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj'
A = make_database(file)
A.check_database()

-----------------------------------------------------------


{'_id': ObjectId('5e5c2233218c6f620ac3b2bc'),
 'name': 'Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj',
 'isvalid': 'no',
 'element': 'Cu',
 'face': '100',
 'unitlength': 2,
 'xc': 'BEEF-vdW',
 'adsorbate': 'CO',
 'numberofads': 1,
 'coverage': 0.25,
 'surfatomnum': 4,
 'E': -18.36377816,
 'bareE': -5.63115633,
 'E_CO': -12.13796648,
 'totaladsE': -0.59465535,
 'aveadsE/suratom': -0.1486638375,
 'aveadsE/ads': -0.59465535,
 'E_int_space': -0.00450823999999983,
 'E_each_ads': None,
 'area': 26.82487613815005,
 'density': 0.03727883009971519,
 'igroups': [0],
 'rgroups': [0],
 'COlengthlis': [1.1539494916714714],
 'converged': 'no',
 'not_moved': 'yes',
 'kept_sites': 'yes',
 'minimum_distance': None,
 'ads_dist2': None,
 'ads_dist3': None}

In [25]:
# file = 'Cu_100_u2_RPBE_no001_CO_n1_d8_all.traj'
# file = 'Cu_100_u2_BEEF-vdW_no001_CO_n1_d8_all.traj'
# # file = 'Cu_100_u2_BEEF-vdW_no010_CO_n1_d8.traj'
# # file = 'Cu_100_u2_RPBE-D2_no001_CO_n1_d8.traj'

atoms = query(file, 'local')
np.max(np.abs(atoms.get_forces()))

0.08157282

In [14]:
atoms.get_potential_energy()

-18.36377816

In [24]:
# file = 'Cu_100_u2_BEEF-vdW_no001_CO_n1_d8.traj'
init_query(file, 'local')

No file named Cu_100_u2_BEEF-vdW_no001_CO_n1_d8_all.traj in init


In [22]:
atoms = query('Cu_100_u2_vdW-DF_no010_CO_n1_d8_all.traj', 'local')
np.max(np.abs(atoms.get_forces()))

0.01037519